In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import time
import csv
import re
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
import os
import urllib.request
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
# To use Tor's SOCKS proxy server with chrome, include the socks protocol in the scheme with the --proxy-server option
# PROXY = "socks5://127.0.0.1:9150" # IP:PORT or HOST:PORT
torexe = os.popen(r'C:\Users\stuti\Tor Browser\Browser\TorBrowser\Tor\tor.exe')
PROXY = "socks5://localhost:9050" # IP:PORT or HOST:PORT
options = webdriver.ChromeOptions()
options.add_argument('--proxy-server=%s' % PROXY)
driver = webdriver.Chrome(chrome_options=options, executable_path=r'C:\chromedriver_win32\chromedriver.exe')

In [ ]:
driver.get('https://www.tripadvisor.in/Hotels-g304551-oa3960-New_Delhi_National_Capital_Territory_of_Delhi-Hotels.html')

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
domain = 'https://www.tripadvisor.in'
#pagination buttons' xpaths
next_page1 = '//*[@id="taplc_main_pagination_bar_dusty_hotels_resp_0"]/div/div/div/a'
next_page = '//*[@id="taplc_main_pagination_bar_dusty_hotels_resp_0"]/div/div/div/a[2]'

#page_list contains no. of pages
page_list = range(int(soup.find("div", {"class": "pageNumbers"}).find_all("a")[-1].get("data-page-number")))

print("Total number of page: {}".format(len(page_list)))

#open csv file to save the data in
with open('./data/hotels.csv', 'a') as csvfile:
    fieldnames = ['hotel_name', 'n_comment', 'n_star', 'url','price']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    counter = 0
    for p in page_list:
        counter+=1
        print('the number of page = {0}/{1}'.format(p+1, len(page_list)))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        hotel_blocks = soup.find_all('div', {"class": "prw_rup prw_meta_hsx_responsive_listing ui_section listItem"})

        for element in hotel_blocks:
            try:
                hotel_name = element.find('div', {"class": "listing_title"}).text
                url = domain+element.find('div', {"class": "listing_title"}).find('a').get('href')
                n_comment = element.find('a', {"class": "review_count"}).text
                n_comment = re.sub('[^0-9,]', "", n_comment).replace(',','')
                n_star = element.find('a', {"data-clicksource":"BubbleRating"}).attrs['alt']
                price = element.find('div', {"class": "price __resizeWatch"}).text
                price=price[1:]
            except AttributeError:
                hotel_name = element.find('div', {"class": "listing_title"}).text
                url = domain+element.find('div', {"class": "listing_title"}).find('a').get('href')
                n_comment = element.find('a', {"class": "review_count"}).text
                n_comment = re.sub('[^0-9,]', "", n_comment).replace(',','')
                n_star = 'None'
                price = element.find('div', {"class": "price __resizeWatch"}).text
                price=price[1:]                

            writer.writerow(
                            {
                                'hotel_name':hotel_name,
                                'n_comment':n_comment,
                                'n_star':n_star,
                                'url':url,
                                'price':price
                            }
                           )              
        if counter<=1:
            driver.find_element_by_xpath(next_page1).click()
            time.sleep(8)
        else:
            driver.find_element_by_xpath(next_page).click()
            time.sleep(8)             